In [ ]:
!pip install transformers datasets scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

🧠 STEP 1: Load the Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("go_emotions")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/9.40k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/2.77M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/350k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/347k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/43410 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5426 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5427 [00:00<?, ? examples/s]

🪄 STEP 2: Preprocess the Text (Tokenize)

In [ ]:
from transformers import BertTokenizer

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True, max_length=128)

# Apply tokenization
encoded_dataset = dataset.map(tokenize, batched=True)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

🎯 STEP 3: Prepare Labels (Multi-label encoding)

In [ ]:
encoded_dataset.reset_format()



In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer

# Fit label binarizer on training labels
mlb = MultiLabelBinarizer()
mlb.fit(dataset['train']['labels'])

def encode_labels(example):
    example['labels'] = mlb.transform([example['labels']])[0].astype(float).tolist()
    return example

encoded_dataset = encoded_dataset.map(encode_labels)



Map:   0%|          | 0/43410 [00:00<?, ? examples/s]

Map:   0%|          | 0/5426 [00:00<?, ? examples/s]

Map:   0%|          | 0/5427 [00:00<?, ? examples/s]

🏋️ STEP 4: Format for PyTorch

In [ ]:
import torch
import datasets

encoded_dataset.set_format(
    type='torch',
    columns=['input_ids', 'attention_mask', 'labels'],
    output_all_columns=False,
    device='cpu'
)

# Explicitly set label dtype to float
encoded_dataset = encoded_dataset.cast_column("labels", datasets.features.Sequence(datasets.Value("float32")))


Casting the dataset:   0%|          | 0/43410 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5426 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/5427 [00:00<?, ? examples/s]

🧠 STEP 5: Load Pretrained BERT + Setup Trainer

In [ ]:
from transformers import BertForSequenceClassification

# BERT model for multi-label classification (we use BCEWithLogitsLoss under the hood)
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=len(mlb.classes_),
    problem_type="multi_label_classification"
)




Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🧪 STEP 6: Training Setup

In [ ]:
import torch
from transformers import TrainingArguments, Trainer
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

def compute_metrics(p):
    preds = (p.predictions > 0.5).astype(int)
    labels = p.label_ids
    return {
        'micro_f1': f1_score(labels, preds, average='micro'),
        'macro_f1': f1_score(labels, preds, average='macro'),
        'accuracy': accuracy_score(labels, preds)
    }

# Set the device here in TrainingArguments
device = "cuda" if torch.cuda.is_available() else "cpu"
from transformers import TrainingArguments

import os
os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir='./results',  # output directory for model checkpoints
    num_train_epochs=3,      # number of training epochs
    per_device_train_batch_size=8,  # batch size for training
    per_device_eval_batch_size=8,   # batch size for evaluation
    logging_dir=None,  # No logging directory for WandB
    logging_steps=500,  # How often to log the metrics
    disable_tqdm=True,  # Disable tqdm progress bar
    report_to="none"    # Disable logging to WandB
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    compute_metrics=compute_metrics
)


🚀 STEP 7: Train the Model

In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
# move model to device
model.to(device)

# Train the model
trainer.train()

{'loss': 0.0822, 'grad_norm': 0.5140990018844604, 'learning_rate': 4.846753884896506e-05, 'epoch': 0.09213193292795283}
{'loss': 0.1017, 'grad_norm': 0.575324535369873, 'learning_rate': 4.693200663349917e-05, 'epoch': 0.18426386585590565}
{'loss': 0.0962, 'grad_norm': 0.32435399293899536, 'learning_rate': 4.539647441803329e-05, 'epoch': 0.2763957987838585}
{'loss': 0.0941, 'grad_norm': 0.6303132772445679, 'learning_rate': 4.3860942202567415e-05, 'epoch': 0.3685277317118113}
{'loss': 0.092, 'grad_norm': 0.8066749572753906, 'learning_rate': 4.232540998710153e-05, 'epoch': 0.46065966463976415}
{'loss': 0.0938, 'grad_norm': 0.5111821889877319, 'learning_rate': 4.078987777163565e-05, 'epoch': 0.552791597567717}
{'loss': 0.0922, 'grad_norm': 0.5107714533805847, 'learning_rate': 3.925434555616977e-05, 'epoch': 0.6449235304956698}
{'loss': 0.0867, 'grad_norm': 0.5344728231430054, 'learning_rate': 3.771881334070389e-05, 'epoch': 0.7370554634236226}
{'loss': 0.0886, 'grad_norm': 0.19836185872554

TrainOutput(global_step=16281, training_loss=0.07182215179889473, metrics={'train_runtime': 4593.9464, 'train_samples_per_second': 28.348, 'train_steps_per_second': 3.544, 'train_loss': 0.07182215179889473, 'epoch': 3.0})

🎉 STEP 8: Evaluate the Model

In [ ]:
# Evaluate on the test set using the trainer
results = trainer.evaluate()

# Print the results
print("Evaluation results:", results)

from sklearn.metrics import precision_score, recall_score

# Get predictions for the test set
predictions = trainer.predict(encoded_dataset["test"])

# Extract predicted labels (assuming binary classification)
pred_labels = (predictions.predictions > 0.5).astype(int)

# Extract true labels
true_labels = predictions.label_ids

precision = precision_score(true_labels, pred_labels, average='micro')
recall = recall_score(true_labels, pred_labels, average='micro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")




{'eval_loss': 0.0900765210390091, 'eval_micro_f1': 0.5654459920671525, 'eval_macro_f1': 0.4463160707461039, 'eval_accuracy': 0.452082565425728, 'eval_runtime': 42.5827, 'eval_samples_per_second': 127.422, 'eval_steps_per_second': 15.945, 'epoch': 3.0}
Evaluation results: {'eval_loss': 0.0900765210390091, 'eval_micro_f1': 0.5654459920671525, 'eval_macro_f1': 0.4463160707461039, 'eval_accuracy': 0.452082565425728, 'eval_runtime': 42.5827, 'eval_samples_per_second': 127.422, 'eval_steps_per_second': 15.945, 'epoch': 3.0}
Precision: 0.7002077082852527
Recall: 0.4793806288513193


Save the Model

In [ ]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
model.save_pretrained("/content/drive/MyDrive/BERT_model")
tokenizer.save_pretrained("/content/drive/MyDrive/BERT_model")

('/content/drive/MyDrive/BERT_model/tokenizer_config.json',
 '/content/drive/MyDrive/BERT_model/special_tokens_map.json',
 '/content/drive/MyDrive/BERT_model/vocab.txt',
 '/content/drive/MyDrive/BERT_model/added_tokens.json')